In [1]:
!pip install ipywidgets

In [2]:
# import pandas as pd
from custom_agents_LG import load_session_data, agent_executor
from llm import get_llama_3dot3_70b_versatile,get_llama_3dot1_8b_instant,get_70b_8192
from loaders_and_chroma_utils import vectorstore, load_and_split_document
from typing_extensions import List, TypedDict
from langgraph.graph import START, StateGraph
# from typing import TypedDict, List, Annotated
# from langgraph.graph import StateGraph, END
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
llm = get_llama_3dot1_8b_instant()

Preprocessing completed successfully
Preprocessing completed successfully
Preprocessing completed successfully


c:\Users\j.mundondo\OneDrive - Statsports\Desktop\statsportsdoc\Projects\frequency_chat_PH\LLM\Langraph\loaders_and_chroma_utils.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


Successfully loaded DOCX!


: 

In [20]:
class State(TypedDict):
    question: str
    context: List[Document]
    player_metrics: str
    answer: str
    player_name: str  # Added to track player name

In [1]:
def verify_document_loading(file_path: str):
    """Load, index, and verify document loading with detailed debugging."""
    print("\nSTEP 1: Loading document...")
    try:
        splits = load_and_split_document(file_path)
        print(f"Successfully split document into {len(splits)} chunks")
        print("\nFirst chunk preview:")
        if splits:
            print(splits[0].page_content[:])
        
        print("\nSTEP 2: Clearing existing vectorstore...")
        existing_ids = vectorstore._collection.get()['ids']
        if existing_ids:
            vectorstore._collection.delete(ids=existing_ids)
        print("Vectorstore cleared")
        
        print("\nSTEP 3: Indexing document chunks...")
        for i, split in enumerate(splits):
            split.metadata['file_id'] = 1
            split.metadata['chunk_id'] = i
        
        vectorstore.add_documents(splits)
        print(f"Added {len(splits)} chunks to vectorstore")
        
        return True
    except Exception as e:
        print(f"ERROR during document processing: {str(e)}")
        import traceback
        traceback.print_exc()
        return False
file_path = r"C:\Users\j.mundondo\OneDrive - Statsports\Desktop\statsportsdoc\Projects\frequency_chat_PH\data\multi_session_hias\Research Paper.docx"
success = verify_document_loading(file_path)


STEP 1: Loading document...
ERROR during document processing: name 'load_and_split_document' is not defined


Traceback (most recent call last):
  File "C:\Users\j.mundondo\AppData\Local\Temp\ipykernel_912\3787832191.py", line 5, in verify_document_loading
    splits = load_and_split_document(file_path)
             ^^^^^^^^^^^^^^^^^^^^^^^
NameError: name 'load_and_split_document' is not defined


In [16]:
# Graph nodes
def extract_player_name(question: str) -> str:
    """Extract player name from the question using the LLM."""
    name_extraction_prompt = """
    Extract the player name(s) from the following question. 
    If no specific player is mentioned, return None.
    Only return the name(s) without any additional text.
    
    Question: {question}
    """
    
    prompt = PromptTemplate.from_template(name_extraction_prompt)
    messages = prompt.invoke({"question": question})
    response = llm.invoke(messages).content.strip()
    
    return response if response.lower() != "none" else None

In [17]:
def get_player_metrics(player_name: str) -> dict:
    """Get metrics for a specific player."""
    return agent_executor.invoke({
        "input": f"Get {player_name}'s metrics as a dictionary format"
    })["output"]

In [ ]:
def setup_initial_state(question: str) -> State:
    """Setup initial state without metrics - moved to separate node."""
    return {
        "question": question,
        "context": [],
        "player_metrics": "",
        "answer": "",
        "player_name": ""
    }